In [ ]:
from __future__ import division
from past.utils import old_div
from proteus import FemTools as ft
from proteus import MeshTools as mt


class PotentialFlowProblem:
    """ PotentialFlowProblem """

    def __init__(self,
                 # DOMAIN AND MESH #
                 domain=None,
                 mesh=None,
                 # INITIAL CONDITIONS #
                 initialConditions=None,
                 # BOUNDARY CONDITIONS #
                 boundaryConditions=None,
                 ):

        self.domain=domain
        self.mesh = mesh
        
        # ***** CREATE SYSTEM PHYSICS OBJECT ***** #
        def r(x):
            return 0.
        def a(x):
            if self.domain.nd == 2:
                return np.array([[1.,0.],
                                 [0.,1.]])
            elif self.domain.nd == 3:
                return np.array([[1.,0.,0.],
                                 [0.,1.,0.],
                                 [0.,0.,1.]])
        physics.coefficients = PoissonEquationCoefficients([a],[r]) 
        physics.name="PoissonDots"

         # ***** CREATE SYSTEM PHYSICS OBJECT ***** #
        self.SystemPhysics = SystemPhysics(ProblemInstance=self)
        self.SystemPhysics.initialConditions = initialConditions
        self.SystemPhysics.boundaryConditions= boundaryConditions        
        
        # ***** CREATE SYSTEM NUMERICS OBJECT ***** #
        self.SystemNumerics = SystemNumerics(ProblemInstance=self)
        
        
        
        
        
        
        
        
        
        # ***** CREATE FINITE ELEMENT SPACES ***** #
        self.FESpace = FESpace().getFESpace()

        # ***** DEFINE PHYSICAL AND NUMERICAL PARAMETERS ***** #
        self.physical_parameters = default_physical_parameters
        self.swe_parameters = default_swe_parameters
        self.GN_swe_parameters = default_GN_swe_parameters
        

    def assert_initialConditions(self, sw_model, initialConditions):
        assert 'water_height' in initialConditions, 'Provide water_height in initialConditions'
        assert 'x_mom' in initialConditions, 'Provide x_mom in initialConditions'
        assert 'y_mom' in initialConditions, 'Provide y_mom in initialConditions'
        if sw_model == 1:  # dispersive SWEs
            assert 'h_times_eta' in initialConditions, 'Provide auxiliary function h_eta in initialConditions'
            assert 'h_times_w' in initialConditions, 'Provide auxiliary function h_w in initialConditions'
            assert 'h_times_beta' in initialConditions, 'Provide auxiliary function h_beta in initialConditions'
    #

    def assert_boundaryConditions(self, sw_model, boundaryConditions):
        # check dirichlet BCs
        assert 'water_height' in boundaryConditions, 'Provide water_height in BCs'
        assert 'x_mom' in boundaryConditions, 'Provide x_mom in BCs'
        assert 'y_mom' in boundaryConditions, 'Provide y_mom in BCs'
        if sw_model == 1:  # dispersive SWEs
            assert 'h_times_eta' in boundaryConditions, 'Provide auxiliary variable h_eta in BCs'
            assert 'h_times_w' in boundaryConditions, 'Provide auxiliary variable h_w in BCs'
            assert 'h_times_beta' in boundaryConditions, 'Provide auxiliary variable h_beta in BCs'


class OutputStepping:
    """
    OutputStepping handles how often the solution is outputted.
    """

    def __init__(self,
                 final_time,
                 dt_init=0.001,
                 dt_output=None,
                 nDTout=None):
        self.final_time = final_time
        self.dt_init = dt_init
        assert not (
            dt_output is None and nDTout is None), "Provide dt_output or nDTout"
        self.dt_output = dt_output
        self.nDTout = nDTout
    #

    def getOutputStepping(self):
        # COMPUTE dt_init #
        dt_init = min(0.1 * self.dt_output, self.dt_init)
        if self.nDTout is None:
            self.nDTout = int(round(old_div(self.final_time, self.dt_output)))
        else:
            self.dt_output = float(self.final_time) / float(self.nDTout)
        #
        return {'final_time': self.final_time,
                'dt_init': dt_init,
                'dt_output': self.dt_output,
                'nDTout': self.nDTout}
#


class FESpace:
    """
    Create FE Spaces.
    """

    def __init__(self):
        pass

    def getFESpace(self):
        basis = ft.C0_AffineLinearOnSimplexWithNodalBasis  # p1 space
        # QUADRATURE RULE #
        elementQuadrature = ft.SimplexGaussQuadrature(2, 3)
        elementBoundaryQuadrature = ft.SimplexGaussQuadrature(1, 3)
        return {'basis': basis,
                'elementQuadrature': elementQuadrature,
                'elementBoundaryQuadrature': elementBoundaryQuadrature}


# ***************************************** #
# ********** PHYSICAL PARAMETERS ********** #
# ***************************************** #
default_physical_parameters = {'gravity': 9.81,
                               'LINEAR_FRICTION': 0,
                               'mannings': 0.0,
                               'gen_start': 0.0,
                               'gen_length': 0.0,
                               'abs_start': 0.0,
                               'abs_length': 0.0,}

# ****************************************** #
# ********** NUMERICAL PARAMETERS ********** #
# ****************************************** #
default_swe_parameters = {'LUMPED_MASS_MATRIX': 0,
                          'cfl': 0.33,
                          'SSPOrder': 3,
                          'cE' : 1.}
default_GN_swe_parameters = {'LUMPED_MASS_MATRIX': 0,
                             'cfl': 0.33,
                             'SSPOrder': 3}